In [1]:
import sys, string
import os
import socket
import time
from time import gmtime, strftime
import operator
import boto3
import json
import findspark
from pyspark.sql import SparkSession
from datetime import datetime


In [2]:
findspark.init()

In [3]:
spark = SparkSession\
        .builder\
        .appName("ethereum-sneha")\
        .getOrCreate()

In [4]:
def good_trans_line(line):
    try:
        fields = line.split(',')
        # print(len(fields))
        # print(fields[0])
        if len(fields) != 7 or fields[0] == 'block_number':
            return False
        else:
            return True
        int(feilds[6])
        # return True
    except:
        return False

# def good_trans_line(line):
#     return True



In [5]:
transaction_file = spark.sparkContext.textFile("./transactions_mini.csv")

In [6]:
transaction_file.collect()[:5]

['block_number,from_address,to_address,value,gas,gas_price,block_timestamp',
 '2280135,0x44130614587f420c4019b7ae9fbf763982cdf2fe,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,1.00E+18,132650,20000000000,1474174806',
 '2280135,0x5af3fa46441acdc8048cff7b5c00b1fb89ffa752,0x5845b8f77109be22c93b4b75a3ad02283b7cc9d8,3.78E+17,21000,20000000000,1474174806',
 '2280135,0x2a65aca4d5fc5b5c859090a6c34d164135398226,0x281dc96ac9d41102ffef09e1410ba4b58e6bb072,1.09E+18,90000,20000000000,1474174806',
 '2280135,0xac4361f56c82ed59d533d45129f407015d84702a,0x41f274c0023f83391de4e0733c609df5a124c3d4,0,1200000,20000000000,1474174806']

In [7]:
clean_transactions = transaction_file.filter(good_trans_line)

In [8]:
clean_transactions.collect()[:2]

['2280135,0x44130614587f420c4019b7ae9fbf763982cdf2fe,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,1.00E+18,132650,20000000000,1474174806',
 '2280135,0x5af3fa46441acdc8048cff7b5c00b1fb89ffa752,0x5845b8f77109be22c93b4b75a3ad02283b7cc9d8,3.78E+17,21000,20000000000,1474174806']

In [9]:
transaction = clean_transactions.map(lambda b: strftime("%Y-%m", gmtime(int(b.split(',')[-1]))))

In [10]:
transaction.collect()[:2]

['2016-09', '2016-09']

In [11]:
transactions_monthly= transaction.map(lambda month: (month, 1)).reduceByKey(lambda a, b: a+b)

In [12]:
transactions_monthly.collect()

[('2016-09', 1000)]

In [13]:
sorted_month_year= transactions_monthly.sortBy(lambda x: x[0])

In [14]:
sorted_month_year.collect()

[('2016-09', 1000)]

In [15]:
monthly_values = clean_transactions.map(lambda b: (time.strftime("%Y-%m", time.gmtime(int(b.split(',')[-1]))), float(b.split(',')[3])))

In [16]:
monthly_sum = monthly_values.aggregateByKey((0.0, 0), lambda accumulator, value: (accumulator[0]+ value, accumulator[1] + 1), lambda accumulator1, accumulator2: (accumulator1[0] + accumulator2[0], accumulator1[1] + accumulator2[1]))

In [17]:
monthly_sum.collect()

[('2016-09', (9.047547928212606e+23, 1000))]

In [18]:
monthly_average = monthly_sum.mapValues(lambda x: round(x[0]/ x[1], 2))

In [19]:
monthly_average.collect()

[('2016-09', 9.047547928212606e+20)]

In [20]:
avg_sorted = monthly_average.sortByKey()

In [21]:
avg_sorted.collect()

[('2016-09', 9.047547928212606e+20)]